In [ ]:
# Install a pip package in the current Jupyter kernel
# or do it using pip from the command prompt
import sys
!{sys.executable} -m pip install ollama

In [ ]:
from ollama import chat
from ollama import ChatResponse

response: ChatResponse = chat(model='deepseek-r1', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])
# or access fields directly from the response object
print(response.message.content)

In [ ]:
import re
def ask_deepseek(input_content, system_prompt, deep_think = True, print_log = True):
    response: ChatResponse = chat(model='deepseek-r1', messages=[
        {'role' : 'system', 'content' : system_prompt},
        {'role': 'user','content': input_content}
    ])
    response_text = response['message']['content']
    if print_log: print(response_text)
    # Extract everything inside <think>...</think> - this is the Deep Think
    think_texts = re.findall(r'<think>(.*?)</think>', response_text, flags=re.DOTALL)
    # Join extracted sections (optional, if multiple <think> sections exist)
    think_texts = "\n\n".join(think_texts).strip()
    # Exclude the Deep Think, and return the response
    clean_response= re.sub(r'<think>.*?</think>', '', response_text, flags=re.DOTALL).strip()

    # Return either the context, or a tuple with the context and deep think
    return clean_response if not deep_think else (clean_response, think_texts)

In [ ]:
import pandas as pd
youtube_comments_df = pd.read_csv(r"YOUR FILE PATH HERE").head(10)
youtube_comments_df

In [ ]:
system_prompt_sentiment = '''You will be provided with a youtube comment. Please rate the Youtube comment on a 
scale of -1 (very negative) to 1 (very positive) where 0 indicates neutral sentiment. Report the scores in increments of 0.1.
Please only answer with the sentiment score and do not include any other word or explanation before or after the sentiment score.
Do not include the '+' symbol in the output if the score is positive, but do inlcude the '-' symbol if the score is negative.
Try your best to look out for humour and sarcasm. Some users may write comments that seem negative on the surface but
are meant to be endearing. The distinction may not be that easy without the context of the video, but do your best!
'''

# Ask deepseek to score each comment and place the result in a new column
#youtube_comments_df['LLM_SENTIMENT'] = youtube_comments_df['TEXT'].apply(lambda comment : ask_deepseek(comment, system_prompt_sentiment, deep_think=False))

# Ask deepseek to score each comment, include the logic, and then place them both in a column
youtube_comments_df[['LLM_SENTIMENT', 'LLM_DEEP_THINK']] = youtube_comments_df['TEXT'].apply(lambda comment : ask_deepseek(comment, system_prompt_sentiment)).apply(pd.Series)

In [ ]:
youtube_comments_df

In [ ]:
pokemon_df = pd.read_excel(r"YOUR FILE PATH HERE")
pokemon_df = pokemon_df.head(9)
pokemon_df

pokemon_df['LLM_INPUT'] = pokemon_df.to_dict(orient='records')
pokemon_df['LLM_INPUT'] = pokemon_df['LLM_INPUT'].astype(str)
pokemon_df['LLM_INPUT'][0]

In [ ]:
system_prompt_summary = """Hi! I am building an app for designed around the popular pokemon games. The app is meant for users to scroll through
each available Pokemon, and look through their various statistics and abilities. Your job is take in an individual Pokemon's data in the form
of a JSON format. From there, I'd like you to write a 3-5 sentence summary about the Pokemon's battle capabilities. Please address
what might be their strengths and weaknesses? Thank you! The output should be a string that formatted as a paragraph that should not exceed
five sentences. Reminder that these stats refer to the Pokemon video games, and not the cards or TV show.

Keep in mind a few things regarding the structure of the input.

+ The name and identifier of the pokemon is in the 'name' field.
+ The main statistics for pokemon battling are in the 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed' fields
+ If a pokemon has a mega evolution denoted in the 'has_mega_evolution' column, then it is worth pointing out
+ The 'evolution' field refers to Pokemon that it evolves into, and should not be confused with the Pokemon currently of interest.
"""
# Ask deepseek to summarize each pokemon's data, include the logic, and then place them both in a column
pokemon_df[['LLM_SUMMARY', 'LLM_DEEP_THINK']] =pokemon_df['LLM_INPUT'].apply(lambda data : ask_deepseek(data, system_prompt_summary)).apply(pd.Series)

In [ ]:
pokemon_df
print(pokemon_df['LLM_SUMMARY'][6])